Will only run in AWS SageMaker

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.35.0
!pip install --disable-pip-version-check -q pandas==1.1.4
!pip install --disable-pip-version-check -q awswrangler==2.7.0
!pip install --disable-pip-version-check -q numpy==1.18.5
!pip install --disable-pip-version-check -q seaborn==0.11.0
!pip install --disable-pip-version-check -q matplotlib===3.3.3

In [ ]:
!aws s3 cp \
  s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv \
  ./womens_clothing_ecommerce_reviews.csv

In [ ]:
import awswrangler as wr
import boto3
from   IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sagemaker
import seaborn as sns
#import csv

In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
df = pd.read_csv('./womens_clothing_ecommerce_reviews.csv',
                 index_col=0)
print(df.shape)
df.head()

### Transform

In [ ]:
df_transformed = df.rename(columns={'Review Text': 'review_body',
                                    'Rating': 'star_rating',
                                    'Class Name': 'product_category'})
df_transformed.drop(
    columns=['Clothing ID', 'Age', 'Title', 'Recommended IND', 
             'Positive Feedback Count', 'Division Name', 'Department Name'],
    inplace=True)
df_transformed.dropna(inplace=True)
df_transformed.shape

In [ ]:
def to_sentiment(star_rating):
    if star_rating in {1, 2}: # negative
        return -1 
    if star_rating == 3:      # neutral
        return 0
    if star_rating in {4, 5}: # positive
        return 1

In [ ]:
df_transformed['sentiment'] = df_transformed['star_rating'].apply(
    lambda star_rating: to_sentiment(star_rating=star_rating))
df_transformed.drop(columns=['star_rating'], inplace=True)
df_transformed = (
    df_transformed.groupby('product_category')
        .filter(lambda reviews: len(reviews) > 10)[
            ['sentiment', 'review_body', 'product_category']])
print(df_transformed.shape)
df_transformed.head()

In [ ]:
df_transformed.to_csv('./womens_clothing_ecommerce_reviews_transformed.csv', 
                      index=False)

In [ ]:
!head -n 5 ./womens_clothing_ecommerce_reviews_transformed.csv

### Register Dataset for Querying

In [ ]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

# Account ID 
sts = (boto3.Session(region_name=region)
       .client(service_name="sts", region_name=region))
account_id = sts.get_caller_identity()['Account']
print('S3 Bucket: {}'.format(bucket))
print('Region: {}'.format(region))
print('Account ID: {}'.format(account_id))

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://s3.console.aws.amazon.com/s3/'
    f'home?region={region}#">Amazon S3 buckets</a></b>'))

In [ ]:
!aws s3 cp ./womens_clothing_ecommerce_reviews_transformed.csv \
  s3://$bucket/data/transformed/womens_clothing_ecommerce_reviews_transformed.csv

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://s3.console.aws.amazon.com/s3/'
    f'buckets/{bucket}?region={region}&prefix=data/transformed/#">'
    'Amazon S3 buckets</a></b>'))

In [ ]:
wr.catalog.create_database(name='dsoaws_deep_learning', exist_ok=True)

In [ ]:
dbs = wr.catalog.get_databases()
for db in dbs:
    print('Database name:', db['Name'])

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://console.aws.amazon.com/glue/'
    f'home?region={region}#catalog:tab=databases">AWS Glue Databases</a>'
    '</b>'))

In [ ]:
wr.catalog.create_csv_table(
    ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
    database='dsoaws_deep_learning', # Replace None
    ### END SOLUTION - DO NOT delete this comment for grading purposes
    path=f's3://{bucket}/data/transformed/', 
    table='reviews',    
    columns_types={'sentiment': 'int',        
                   'review_body': 'string',
                   'product_category': 'string'},
    mode='overwrite',
    skip_header_line_count=1,
    sep=',')

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://console.aws.amazon.com/glue/'
    f'home?region={region}#">AWS Glue Catalog</a></b>'))

In [ ]:
table = wr.catalog.table(database='dsoaws_deep_learning', table='reviews')
table

In [ ]:
wr.athena.create_athena_bucket()

### Viz

In [ ]:
database_name = 'dsoaws_deep_learning'
table_name = 'reviews'

In [ ]:
sns.set_style = 'seaborn-whitegrid'
sns.set(rc={'font.style': 'normal',
            'axes.facecolor": 'white',
            'grid.color': '.8',
            'grid.linestyle': '-',
            'figure.facecolor': 'white',
            'figure.titlesize': 20,
            'text.color': 'black',
            'xtick.color': 'black',
            'ytick.color': 'black',
            'axes.labelcolor': 'black',
            'axes.grid': True,
            'axes.labelsize': 10,
            'xtick.labelsize': 10,
            'font.size': 10,
            'ytick.labelsize': 10})

In [ ]:
statement_count_by_sentiment = '''
SELECT sentiment, COUNT(sentiment) AS count_sentiment
FROM reviews
GROUP BY sentiment
ORDER BY sentiment
'''
print(statement_count_by_sentiment)

In [ ]:
df_count_by_sentiment = wr.athena.read_sql_query(
    sql=statement_count_by_sentiment, database=database_name)
df_count_by_sentiment

In [ ]:
df_count_by_sentiment.plot(
    kind='bar', x='sentiment', y='count_sentiment', rot=0);

In [ ]:
# Replace all None
### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
statement_count_by_category = f'''
SELECT product_category, COUNT(sentiment) AS count_sentiment
FROM {table_name}
GROUP BY product_category 
ORDER BY count_sentiment DESC
'''
### END SOLUTION - DO NOT delete this comment for grading purposes
print(statement_count_by_category)

In [ ]:
%%time
df_count_by_category = wr.athena.read_sql_query(
    sql=statement_count_by_category,
    database=database_name)
df_count_by_category

# EXPECTED OUTPUT
# Dresses: 6145
# Knits: 4626
# Blouses: 2983
# Sweaters: 1380
# Pants: 1350
# ...

In [ ]:
statement_avg_by_category = f'''
SELECT product_category, AVG(sentiment) AS avg_sentiment
FROM {table_name} 
GROUP BY product_category 
ORDER BY avg_sentiment DESC
'''
print(statement_avg_by_category)

In [ ]:
%%time
df_avg_by_category = wr.athena.read_sql_query(
    sql=statement_avg_by_category,
    database=database_name)

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://s3.console.aws.amazon.com/s3/'
    f'buckets/aws-athena-query-results-{account_id}-{region}?'
    f'region={region}">Amazon S3 buckets</a></b>'))

In [ ]:
df_avg_by_category

In [ ]:
def show_values_barplot(axs, space):
    def _show_on_plot(ax):
        for p in ax.patches:
            _x = p.get_x() + p.get_width() + float(space)
            _y = p.get_y() + p.get_height()
            value = round(float(p.get_width()),2)
            ax.text(_x, _y, value, ha='left')

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_plot(ax)
    else:
        _show_on_plot(axs)

In [ ]:
barplot = sns.barplot(data=df_avg_by_category, 
                      y='product_category',
                      x='avg_sentiment', 
                      color='b', 
                      saturation=1)

# Set the size of the figure
sns.set(rc={'figure.figsize': (15.0, 10.0)})
    
# Set title and x-axis ticks 
plt.title('Average sentiment by product category')
#plt.xticks([-1, 0, 1], ['Negative', 'Neutral', 'Positive'])

# Helper code to show actual values afters bars 
show_values_barplot(barplot, 0.1)

plt.xlabel('Average sentiment')
plt.ylabel('Product category')

plt.tight_layout()
# Do not change the figure name - it is used for grading purposes!
plt.savefig('avg_sentiment_per_category.png', dpi=300)

# Show graphic
plt.show(barplot)

In [ ]:
sess.upload_data(path='avg_sentiment_per_category.png', 
                 bucket=bucket, 
                 key_prefix='images')

In [ ]:
display(HTML(
    f'<b>Review <a target="top" href="https://s3.console.aws.amazon.com/s3/'
    f'home?region={region}">Amazon S3 buckets</a></b>'))

In [ ]:
statement_count_by_category_desc = f'''
SELECT product_category, COUNT(*) AS count_reviews 
FROM {table_name}
GROUP BY product_category 
ORDER BY count_reviews DESC
'''
print(statement_count_by_category_desc)

In [ ]:
%%time
df_count_by_category_desc = wr.athena.read_sql_query(
    sql=statement_count_by_category_desc, database=database_name)

In [ ]:
max_sentiment = df_count_by_category_desc.count_reviews.max()
print('Highest number of reviews (in a single category):', max_sentiment)

In [ ]:
# Create seaborn barplot
barplot = sns.barplot(
    ### BEGIN SOLUTION - DO NOT delete this comment for grading purposes
    data=df_count_by_category, # Replace None
    y='product_category', # Replace None
    x='count_sentiment', # Replace None
    ### END SOLUTION - DO NOT delete this comment for grading purposes
    color='b',
    saturation=1)

# Set the size of the figure
sns.set(rc={'figure.figsize': (15.0, 10.0)})
    
# Set title
plt.title('Number of reviews per product category')
plt.xlabel('Number of reviews')
plt.ylabel('Product category')
plt.tight_layout()

# Do not change the figure name - it is used for grading purposes!
plt.savefig('n_reviews_per_category.png', dpi=300)

# Show the barplot
plt.show(barplot)

In [ ]:
sess.upload_data(path='n_reviews_per_category.png', 
                 bucket=bucket, 
                 key_prefix='images')

In [ ]:
statement_count_by_category_and_sentiment = f'''
SELECT product_category, sentiment, COUNT(*) AS count_reviews
FROM {table_name}
GROUP BY  product_category, sentiment
ORDER BY  product_category ASC, sentiment DESC, count_reviews
'''
print(statement_count_by_category_and_sentiment)

In [ ]:
%%time
df_count_by_category_and_sentiment = wr.athena.read_sql_query(
    sql=statement_count_by_category_and_sentiment,
    database=database_name)

In [ ]:
# Create grouped dataframes by category and by sentiment
grouped_category = df_count_by_category_and_sentiment.groupby(
    'product_category')
grouped_star = df_count_by_category_and_sentiment.groupby('sentiment')

# Create sum of sentiments per star sentiment
df_sum = df_count_by_category_and_sentiment.groupby(['sentiment']).sum()

# Calculate total number of sentiments
total = df_sum.count_reviews.sum()
print('Total number of reviews:', total)

In [ ]:
distribution = {}
count_reviews_per_star = []
i = 0

for category, sentiments in grouped_category:
    count_reviews_per_star = []
    for star in sentiments['sentiment']:
        count_reviews_per_star.append(sentiments.at[i, 'count_reviews'])
        i += 1
    distribution[category] = count_reviews_per_star
distribution    

In [ ]:
df_distribution_pct = pd.DataFrame(distribution).transpose().apply(
    lambda n_sentiments: n_sentiments / sum(n_sentiments) * 100, 
    axis=1)
df_distribution_pct.columns=['1', '0', '-1']
df_distribution_pct

In [ ]:
categories = df_distribution_pct.index

# Plot bars
plt.figure(figsize=(10, 5))
df_distribution_pct.plot(kind='barh', 
                         stacked=True, 
                         edgecolor='white',
                         width=1.0,
                         color=['green', 'orange', 'blue'])
plt.title('Distribution of reviews per sentiment per category', 
          fontsize='16')
plt.legend(bbox_to_anchor=(1.04, 1), 
           loc='upper left',
           labels=['Positive', 'Neutral', 'Negative'])
plt.xlabel('% Breakdown of sentiments', fontsize='14')
plt.gca().invert_yaxis()
plt.tight_layout()

# Do not change the figure name - it is used for grading purposes!
plt.savefig('distribution_sentiment_per_category.png', dpi=300)
plt.show();

In [ ]:
sess.upload_data(path='distribution_sentiment_per_category.png', 
                 bucket=bucket, 
                 key_prefix='images')

In [ ]:
statement_n_words = f'''
SELECT CARDINALITY(SPLIT(review_body, ' ')) as n_words
FROM {table_name}'''
print(statement_n_words)

In [ ]:
%%time
df_n_words = wr.athena.read_sql_query(sql=statement_n_words,
                                      database=database_name)

In [ ]:
summary = df_n_words.n_words.describe(
    percentiles=[
        0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00])
summary

In [ ]:
df_n_words.n_words.plot.hist(
    xticks=[0, 16, 32, 64, 128, 256], 
    bins=100, 
    range=[0, 256]
).axvline(x=summary['100%'], c='red')
plt.xlabel('Words number', fontsize='14')
plt.ylabel('Frequency', fontsize='14')
plt.savefig('distribution_n_words_per_review.png', dpi=300)
plt.show()

In [ ]:
sess.upload_data(path='distribution_n_words_per_review.png', 
                 bucket=bucket, 
                 key_prefix='images')